<a href="https://colab.research.google.com/github/kwasganguly/Hypernetwork-using-Keras-on-Tensorflow-2.0/blob/master/tf_keras_for_Researchers_Crash_Course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly-gpu-2.0-preview

    100% |████████████████████████████████| 337.4MB 46kB/s 
    100% |████████████████████████████████| 3.0MB 8.7MB/s 
    100% |████████████████████████████████| 61kB 24.0MB/s 
    100% |████████████████████████████████| 358kB 12.2MB/s 


In [2]:
import tensorflow as tf

tf.__version__

'2.0.0-dev20190323'

# tf.keras for Researchers:

This notebook is a crash course on everything you need to know to use TensorFlow 2.0 for deep learning research.

1) The first class you need to know is `Layer`. A Layer encapsulates a state (weights) and some computation (defined in the `call` method).


In [0]:
from tensorflow.keras.layers import Layer


class Linear(Layer):
  """y = w.x + b"""

  def __init__(self, units=32, input_dim=32):
      super(Linear, self).__init__()
      w_init = tf.random_normal_initializer()
      self.w = tf.Variable(
          initial_value=w_init(shape=(input_dim, units), dtype='float32'),
          trainable=True)
      b_init = tf.zeros_initializer()
      self.b = tf.Variable(
          initial_value=b_init(shape=(units,), dtype='float32'),
          trainable=True)

  def call(self, inputs):
      return tf.matmul(inputs, self.w) + self.b


# Instantiate our layer.
linear_layer = Linear(4, 2)

# The layer can be treated as a function.
# Here we call it on some data.
y = linear_layer(tf.ones((2, 2)))
assert y.shape == (2, 4)

# Weights are automatically tracked under the `weights` property.
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

2) The `add_weight` method gives you a shortcut for creating weights.

3) It’s good practice to create weights in a separate `build` method, called lazily with the shape of the first inputs seen by your layer. Here, this pattern prevents us from having to specify `input_dim` in the constructor:


In [0]:
class Linear(Layer):
  """y = w.x + b"""

  def __init__(self, units=32):
      super(Linear, self).__init__()
      self.units = units

  def build(self, input_shape):
      self.w = self.add_weight(shape=(input_shape[-1], self.units),
                               initializer='random_normal',
                               trainable=True)
      self.b = self.add_weight(shape=(self.units,),
                               initializer='random_normal',
                               trainable=True)

  def call(self, inputs):
      return tf.matmul(inputs, self.w) + self.b


# Instantiate our lazy layer.
linear_layer = Linear(4)

# This will also call `build(input_shape)` and create the weights.
y = linear_layer(tf.ones((2, 2)))

4) You can automatically retrieve the gradients of the weights of a layer by calling it inside a `GradientTape`.  Using these gradients, you can update the weights of the layer, either manually, or using an optimizer object. Of course, you can modify the gradients before using them, if you need to.

In [6]:
# Prepare a dataset.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype('float32') / 255, y_train))
dataset = dataset.shuffle(buffer_size=1024).batch(64)

# Instantiate our linear layer (defined above) with 10 units.
linear_layer = Linear(10)

# Instantiate a logistic loss function that expects integer targets.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

# Iterate over the batches of the dataset.
for step, (x, y) in enumerate(dataset):
  
  # Open a GradientTape.
  with tf.GradientTape() as tape:

    # Forward pass.
    logits = linear_layer(x)

    # Loss value for this batch.
    loss = loss_fn(y, logits)
     
  # Get gradients of weights wrt the loss.
  gradients = tape.gradient(loss, linear_layer.trainable_weights)
  
  # Update the weights of our linear layer.
  optimizer.apply_gradients(zip(gradients, linear_layer.trainable_weights))
  
  # Logging.
  if step % 100 == 0:
    print(step, float(loss))

11493376/11490434 [==============================] - 0s 0us/step
0 2.442819356918335
100 2.235952377319336
200 2.1712589263916016
300 2.0394351482391357
400 2.0120344161987305
500 1.863042950630188
600 1.8589389324188232
700 1.745722770690918
800 1.706221580505371
900 1.7238972187042236


5) Weights created by layers can be either trainable or non-trainable. They're exposed in `trainable_weights` and `non_trainable_weights`. Here's a layer with a non-trainable weight:

In [8]:
class ComputeSum(Layer):
  """Returns the sum of the inputs."""

  def __init__(self, input_dim):
      super(ComputeSum, self).__init__()
      # Create a non-trainable weight.
      self.total = tf.Variable(initial_value=tf.zeros((input_dim,)),
                               trainable=False)

  def call(self, inputs):
      self.total.assign_add(tf.reduce_sum(inputs, axis=0))
      return self.total  

my_sum = ComputeSum(2)
x = tf.ones((2, 2))

y = my_sum(x)
print(y.numpy())  # [2. 2.]

y = my_sum(x)
print(y.numpy())  # [4. 4.]

assert my_sum.weights == [my_sum.total]
assert my_sum.non_trainable_weights == [my_sum.total]
assert my_sum.trainable_weights == []

[2. 2.]
[4. 4.]


6) Layers can be recursively nested to create bigger computation blocks. Each layer will track the weights of its sublayers (both trainable and non-trainable).

In [0]:
# Let's reuse the Linear class
# with a `build` method that we defined above.

class MLP(Layer):
    """Simple stack of Linear layers."""

    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(10)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)

mlp = MLP()

# The first call to the `mlp` object will create the weights.
y = mlp(tf.ones(shape=(3, 64)))

# Weights are recursively tracked.
assert len(mlp.weights) == 6

7) Layers can create losses during the forward pass. This is especially useful for regularization losses. The losses created by sublayers are recursively tracked by the parent layers.

In [10]:
class ActivityRegularization(Layer):
  """Layer that creates an activity sparsity regularization loss."""
  
  def __init__(self, rate=1e-2):
    super(ActivityRegularization, self).__init__()
    self.rate = rate
  
  def call(self, inputs):
    # We use `add_loss` to create a regularization loss
    # that depends on the inputs.
    self.add_loss(self.rate * tf.reduce_sum(inputs))
    return inputs


# Let's use the loss layer in a MLP block.

class SparseMLP(Layer):
  """Stack of Linear layers with a sparsity regularization loss."""

  def __init__(self):
      super(SparseMLP, self).__init__()
      self.linear_1 = Linear(32)
      self.regularization = ActivityRegularization(1e-2)
      self.linear_3 = Linear(10)

  def call(self, inputs):
      x = self.linear_1(inputs)
      x = tf.nn.relu(x)
      x = self.regularization(x)
      return self.linear_3(x)
    

mlp = SparseMLP()
y = mlp(tf.ones((10, 10)))

print(mlp.losses)  # List containing one float32 scalar

[<tf.Tensor: id=73567, shape=(), dtype=float32, numpy=0.23226993>]


8) These losses are cleared by the top-level layer at the start of each forward pass -- they don't accumulate. `layer.losses` always contain only the losses created during the *last* forward pass. You would typically use these losses by summing them before computing your gradients when writing a training loop.

In [11]:
# Losses correspond to the *last* forward pass.
mlp = SparseMLP()
mlp(tf.ones((10, 10)))
assert len(mlp.losses) == 1
mlp(tf.ones((10, 10)))
assert len(mlp.losses) == 1  # No accumulation.

# Let's demonstrate how to use these losses in a training loop.

# Prepare a dataset.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype('float32') / 255, y_train))
dataset = dataset.shuffle(buffer_size=1024).batch(64)

# A new MLP.
mlp = SparseMLP()

# Loss and optimizer.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

for step, (x, y) in enumerate(dataset):
  with tf.GradientTape() as tape:

    # Forward pass.
    logits = mlp(x)

    # External loss value for this batch.
    loss = loss_fn(y, logits)
    
    # Add the losses created during the forward pass.
    loss += sum(mlp.losses)
     
    # Get gradients of weights wrt the loss.
    gradients = tape.gradient(loss, mlp.trainable_weights)
  
  # Update the weights of our linear layer.
  optimizer.apply_gradients(zip(gradients, mlp.trainable_weights))
  
  # Logging.
  if step % 100 == 0:
    print(step, float(loss))

0 6.312371253967285
100 2.5329384803771973
200 2.3832437992095947
300 2.3720664978027344
400 2.350461483001709
500 2.3381080627441406
600 2.3193042278289795
700 2.3529601097106934
800 2.3268256187438965
900 2.3302135467529297


9) Running eagerly is great for debugging, but you will get better performance by compiling your computation into static graphs. Static graphs are a researcher's best friends. You can compile any function by wrapping it in a `tf.function` decorator.

In [12]:
# Prepare our layer, loss, and optimizer.
mlp = MLP()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

# Create a training step function.

@tf.function  # Make it fast.
def train_on_batch(x, y):
  with tf.GradientTape() as tape:
    logits = mlp(x)
    loss = loss_fn(y, logits)
    gradients = tape.gradient(loss, mlp.trainable_weights)
  optimizer.apply_gradients(zip(gradients, mlp.trainable_weights))
  return loss

# Prepare a dataset.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype('float32') / 255, y_train))
dataset = dataset.shuffle(buffer_size=1024).batch(64)
  
for step, (x, y) in enumerate(dataset):
  loss = train_on_batch(x, y)
  if step % 100 == 0:
    print(step, float(loss))

0 2.306180000305176
100 2.301784038543701
200 2.3096704483032227
300 2.302459239959717
400 2.3040270805358887
500 2.3009305000305176
600 2.301976203918457
700 2.2945547103881836
800 2.2995829582214355
900 2.310793399810791


10) Some layers, in particular the `BatchNormalization` layer and the `Dropout` layer, have different behaviors during training and inference. For such layers, it is standard practice to expose a `training` (boolean) argument in the `call` method.

By exposing this argument in `call`, you enable the built-in training and evaluation loops (e.g. `fit`) to correctly use the layer in training and inference.

In [0]:
class Dropout(Layer):
  
  def __init__(self, rate):
    super(Dropout, self).__init__()
    self.rate = rate

  @tf.function
  def call(self, inputs, training=None):
    # Note that the tf.function decorator enables use
    # to use imperative control flow like this `if`,
    # while defining a static graph!
    if training:
      return tf.nn.dropout(inputs, rate=self.rate)
    return inputs
  

class MLPWithDropout(Layer):

  def __init__(self):
      super(MLPWithDropout, self).__init__()
      self.linear_1 = Linear(32)
      self.dropout = Dropout(0.5)
      self.linear_3 = Linear(10)

  def call(self, inputs, training=None):
      x = self.linear_1(inputs)
      x = tf.nn.relu(x)
      x = self.dropout(x, training=training)
      return self.linear_3(x)
    
mlp = MLPWithDropout()
y_train = mlp(tf.ones((2, 2)), training=True)
y_test = mlp(tf.ones((2, 2)), training=False)


11) You have many built-in layers available, from `Dense` to `Conv2D` to `LSTM` to fancier ones like `Conv2DTranspose` or `ConvLSTM2D`. Be smart about reusing built-in functionality.

12) To build deep learning models, you don't have to use object-oriented programming all the time. All layers we've seen so far can also be composed functionally, like this (we call it the "Functional API"):

In [0]:
# We use an `Input` object to describe the shape and dtype of the inputs.
# This is the deep learning equivalent of *declaring a type*.
# The shape argument is per-sample; it does not include the batch size.
# The functional API focused on defining per-sample transformations.
# The model we create will automatically batch the per-sample transformations,
# so that it can be called on batches of data.
inputs = tf.keras.Input(shape=(16,))

# We call layers on these "type" objects
# and they return updated types (new shapes/dtypes).
x = Linear(32)(inputs) # We are reusing the Linear layer we defined earlier.
x = Dropout(0.5)(x) # We are reusing the Dropout layer we defined earlier.
outputs = Linear(10)(x)

# A functional `Model` can be defined by specifying inputs and outputs.
# A model is itself a layer like any other.
model = tf.keras.Model(inputs, outputs)

# A functional model already has weights, before being called on any data.
# That's because we defined its input shape in advance (in `Input`).
assert len(model.weights) == 4

# Let's call our model on some data, for fun.
y = model(tf.ones((2, 16)))
assert y.shape == (2, 10)

# You can pass a `training` argument in `__call__`
# (it will get passed down to the Dropout layer).
y = model(tf.ones((2, 16)), training=True)

The Functional API tends to be more concise than subclassing, and provides a few other advantages (generally the same advantages that functional, typed languages provide over untyped OO development). However, it can only be used to define DAGs of layers -- recursive networks should be defined as `Layer` subclasses instead.

Learn more about the Functional API [here](https://www.tensorflow.org/alpha/guide/keras/functional).

In your research workflows, you may often find yourself mix-and-matching OO models and Functional models.

### That's all you need to get started with reimplementing most deep learning research papers in TensorFlow 2.0 and Keras!

## Now for a quick research example: hypernetworks.

Let's put these concepts into practice with a simple end-to-end example.

A [hypernetwork](http://blog.otoro.net/2016/09/28/hyper-networks/) is a deep neural network whose weights are generated by another network (usually smaller).

Let's implement a really trivial hypernetwork: we'll take the `Linear` layer we defined earlier, and we'll use it to generate the weights of... another `Linear` layer.

In [15]:
input_dim = 784
classes = 10

# The model we'll actually use (the hypernetwork).
outer_model = Linear(classes)

# It doesn't need to create its own weights, so let's mark it as already built.
# That way, calling `outer_model` won't create new variables.
outer_model.built = True

# The model that generates the weights of the model above.
inner_model = Linear(input_dim * classes + classes)

# Loss and optimizer.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

# Prepare a dataset.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype('float32') / 255, y_train))

# We'll use a batch size of 1 for this experiment.
dataset = dataset.shuffle(buffer_size=1024).batch(1)
  
losses = []  # Keep track of the losses over time.
for step, (x, y) in enumerate(dataset):
  with tf.GradientTape() as tape:

    # Predict weights for the outer model.
    weights_pred = inner_model(x)

    # Reshape them to the expected shapes for w and b for the outer model.
    w_pred = tf.reshape(weights_pred[:, :-classes], (input_dim, classes))
    b_pred = tf.reshape(weights_pred[:, -classes:], (classes,))
    
    # Set the weight predictions as the weight variables on the outer model.
    outer_model.w = w_pred
    outer_model.b = b_pred
    
    # Inference on the outer model.
    preds = outer_model(x)
    loss = loss_fn(y, preds)

  # Train only inner model.
  grads = tape.gradient(loss, inner_model.trainable_weights)
  optimizer.apply_gradients(zip(grads, inner_model.trainable_weights))
  
  # Logging.
  losses.append(float(loss))
  if step % 100 == 0:
    print(step, sum(losses) / len(losses))
    
  # Stop after 1000 steps.
  if step >= 1000:
    break

0 12.5589017868042
100 4.768053405829208
200 3.288215119380115
300 2.944460560511529
400 2.47045986208493
500 2.3251449641156823
600 2.176233513761083
700 2.0036057443016873
800 1.8577934748203133
900 1.7495915554592563
1000 1.7572440114531558
